**Fundamentals of DataScience Project Phase 2**

**Student:** Kimia Mohsenian 

**Student ID:** 97110671

**Date:** December 2022

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pickle
from itertools import islice
import seaborn as sns
import nltk
import html
import re
import string
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score
from string import digits

## Laoding the data:

In [ ]:
DataPath = 'gdrive/MyDrive/DataScience/Project/dataset/training.1600000.processed.noemoticon.csv'

In [ ]:
data = pd.read_csv(DataPath, encoding='latin-1')

In [ ]:
data.head(4)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# setting column names for the dataframe
data.columns = ['label', 'id', 'date_created', 'query_status', 'name','tweet']

In [ ]:
data.head(4)

,label,id,date_created,query_status,name,tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
print('Number of total tweets: ', len(data))

Number of total tweets:  1599999


## 1. Cleaning tweets

In [ ]:
pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=f618df515078a21feb1b61885b6290b40ef6fde2e16478b7d23611b487ce1c0e
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [ ]:
import emoji

In [ ]:
# Removing duplicates
new_data = data.drop_duplicates('tweet', keep='first')

In [ ]:
print('Number of duplicate tweets removed: ', len(data)-len(new_data))

Number of duplicate tweets removed:  18534


In [ ]:
def tweet_cleaner(tweet):
    # removing hashtag signs
    tweet = tweet.replace("#", "").replace("_", " ")
    # removing @ sign
    tweet = re.sub("@[A-Za-z0-9]+","",tweet)
    # removing links                           
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet)   
    tweet = " ".join(tweet.split())
    # removing emojis
    tweet = ''.join(c for c in tweet if c not in emoji.EMOJI_DATA)
    # removing captalization
    tweet = tweet.lower().strip()
    # removing numbers
    remove_digits = str.maketrans('', '', digits)
    tweet = tweet.translate(remove_digits)
    # removing punctuation marks
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    return tweet

In [ ]:
new_data['tweet'] = new_data['tweet'].map(lambda tweet: tweet_cleaner(tweet))

<ipython-input-14-034dca2325b5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['tweet'] = new_data['tweet'].map(lambda tweet: tweet_cleaner(tweet))


In [ ]:
new_data.head(4)

,label,id,date_created,query_status,name,tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he cant update his facebook by t...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,i dived many times for the ball managed to sav...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,no its not behaving at all im mad why am i her...


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
new_data['tokenized_text'] = new_data['tweet'].apply(word_tokenize) 

<ipython-input-18-1d53fa9b848d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['tokenized_text'] = new_data['tweet'].apply(word_tokenize)


In [ ]:
new_data.head(4)

,label,id,date_created,query_status,name,tweet,tokenized_text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he cant update his facebook by t...,"[is, upset, that, he, cant, update, his, faceb..."
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,i dived many times for the ball managed to sav...,"[i, dived, many, times, for, the, ball, manage..."
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"[my, whole, body, feels, itchy, and, like, its..."
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,no its not behaving at all im mad why am i her...,"[no, its, not, behaving, at, all, im, mad, why..."


## 2. Lemmatization

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize_words(words):
  words = [lemmatizer.lemmatize(word,pos='a') for word in words]
  return words

In [ ]:
new_data['lemma_words'] = new_data['tokenized_text'].apply(lemmatize_words)

<ipython-input-26-0fa4896a1bdb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['lemma_words'] = new_data['tokenized_text'].apply(lemmatize_words)


In [ ]:
new_data.head(4)

,label,id,date_created,query_status,name,tweet,tokenized_text,lemma_words,cleaned_tweets
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he cant update his facebook by t...,"[is, upset, that, he, cant, update, his, faceb...","[is, upset, that, he, cant, update, his, faceb...",is upset that he cant update his facebook by t...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,i dived many times for the ball managed to sav...,"[i, dived, many, times, for, the, ball, manage...","[i, dived, many, times, for, the, ball, manage...",i dived many times for the ball managed to sav...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"[my, whole, body, feels, itchy, and, like, its...","[my, whole, body, feels, itchy, and, like, its...",my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,no its not behaving at all im mad why am i her...,"[no, its, not, behaving, at, all, im, mad, why...","[no, its, not, behaving, at, all, im, mad, why...",no its not behaving at all im mad why am i her...


In [ ]:
cleaned_tweets = []
for lemma_words in list(new_data['lemma_words']):
  cleaned_tweets.append(' '.join(lemma_words))
new_data['cleaned_tweets'] = cleaned_tweets

<ipython-input-37-c5542e16dcad>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['cleaned_tweets'] = cleaned_tweets


In [ ]:
new_data.head(4)

,label,id,date_created,query_status,name,tweet,tokenized_text,lemma_words,cleaned_tweets
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he cant update his facebook by t...,"[is, upset, that, he, cant, update, his, faceb...","[is, upset, that, he, cant, update, his, faceb...",is upset that he cant update his facebook by t...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,i dived many times for the ball managed to sav...,"[i, dived, many, times, for, the, ball, manage...","[i, dived, many, times, for, the, ball, manage...",i dived many times for the ball managed to sav...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"[my, whole, body, feels, itchy, and, like, its...","[my, whole, body, feels, itchy, and, like, its...",my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,no its not behaving at all im mad why am i her...,"[no, its, not, behaving, at, all, im, mad, why...","[no, its, not, behaving, at, all, im, mad, why...",no its not behaving at all im mad why am i her...


## 3. Feature Extraction

#### BoW

In [ ]:
BoW_FeatureExtractor = CountVectorizer(max_df=0.90, min_df=2, max_features=1500, stop_words='english')

In [ ]:
BoW_Features = BoW_FeatureExtractor.fit_transform(new_data['cleaned_tweets'])

In [ ]:
print(np.shape(BoW_Features))

(1581465, 1500)


#### TF-IDF

In [ ]:
TFIDF_FeatureExtractor = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1500, stop_words='english')

In [ ]:
TFIDF_Features = TFIDF_FeatureExtractor.fit_transform(new_data['cleaned_tweets'])

In [ ]:
print(np.shape(TFIDF_Features))

(1581465, 1500)


## 4. Logistic Regression

### BoW

In [ ]:
# train-test split
BoW_Xtrain, BoW_Xtest, BoW_Ytrain, BoW_Ytest = train_test_split(BoW_Features , list(new_data['label']) , train_size=0.75, random_state=42 )

In [ ]:
np.shape(BoW_Xtrain)

(1186098, 1500)

In [ ]:
np.shape(BoW_Xtest)

(395367, 1500)

In [89]:
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

In [90]:
# training the logistic regression model on the train set
model.fit(BoW_Xtrain , BoW_Ytrain)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [107]:
# model performance on train set
TRAINpredicted_labels_BoW = model.predict(BoW_Xtrain)
TrainAcc_BoW = accuracy_score(TRAINpredicted_labels_BoW, BoW_Ytrain)
print('Train Accuracy of logistic regression model with BoW features is: ', np.round(TrainAcc_BoW, 4))

Train Accuracy of logistic regression model with BoW features is:  0.7469


In [108]:
# testing
TESTpredicted_labels_BoW = model.predict(BoW_Xtest)
TestAcc_BoW = accuracy_score(TESTpredicted_labels_BoW, BoW_Ytest)
print('Test Accuracy of logistic regression model with BoW features is: ', np.round(TestAcc_BoW, 4))

Test Accuracy of logistic regression model with BoW features is:  0.7467


### TF-IDF

In [102]:
# train-test split
TFIDF_Xtrain, TFIDF_Xtest, TFIDF_Ytrain, TFIDF_Ytest = train_test_split(TFIDF_Features , list(new_data['label']) , train_size=0.75, random_state=42 )

In [103]:
# training the logistic regression model on the train set
model.fit(TFIDF_Xtrain , TFIDF_Ytrain)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [110]:
# model performance on train set
TRAINpredicted_labels_TFIDF = model.predict(TFIDF_Xtrain)
TrainAcc_TFIDF = accuracy_score(TRAINpredicted_labels_TFIDF, TFIDF_Ytrain)
print('Train Accuracy of logistic regression model with TF-IDF features is: ', np.round(TrainAcc_TFIDF, 4))

Train Accuracy of logistic regression model with TF-IDF features is:  0.7485


In [111]:
# testing
TESTpredicted_labels_TFIDF = model.predict(TFIDF_Xtest)
TestAcc_TFIDF = accuracy_score(TESTpredicted_labels_TFIDF, TFIDF_Ytest)
print('Test Accuracy of logistic regression model with TF-IDF features is: ', np.round(TestAcc_TFIDF, 4))

Test Accuracy of logistic regression model with TF-IDF features is:  0.7481


We can see that classification with TF-IDF results in better model performance, but with a slight difference.

## 5. Pre-trained library for sentiment analysis:

For this part, I use "pysentimiento" which is a transformer-based library for SocialNLP tasks.

The output of the prediction using "sentiment analyer" of this model would be the predicted sentiment category (Positive, Negative or Neutral) along with the probabilities.

In [119]:
pip install pysentimiento

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [120]:
from pysentimiento import create_analyzer

In [121]:
analyzer = create_analyzer(task="sentiment", lang="es")

Downloading:   0%|          | 0.00/925 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/435M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/334 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/858k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [126]:
tweets_to_be_analyzied = list(new_data['cleaned_tweets'])[0:150000]

In [ ]:
predicted_sentiment = analyzer.predict(tweets_to_be_analyzied)

  0%|          | 0/4688 [00:00<?, ?ba/s]

***** Running Prediction *****
  Num examples = 150000
  Batch size = 32
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.


The run-time of the program was so high! So I couldn't obtain the final results (even when I reduced number of sampels)

But the procedure was completely described.